# Controlled-Z Gate

*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
In this tutorial, we will implement a controlled-Z gate using Quanlse Cloud Service. The outline of this tutorial is as follows:
- Preparation
- Introduction
- Construct Hamiltonian
- Generate and optimize pulse via Quanlse Cloud Service
- Summary

## Preparation
After you have successfully installed Quanlse on your device, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import Hamiltonian-related modules 
from Quanlse.Utils import Hamiltonian as qham
from Quanlse.Utils.Operator import create, destroy, number, duff

# Import the optimizer for controlled-z gate
from Quanlse.remoteOptimizer import remoteOptimizeCz

# Import tools for result analysis
from Quanlse.Utils.Tools import project

# Import numpy and math
from numpy import round
from math import pi

## Introduction

Theoretically, the controlled-Z gate (CZ gate) is a two-qubit quantum gate that adds a phase of $e^{i\pi}$ to the target qubit if the control qubit and target qubit's state is $|11\rangle$. The physical implementation of the CZ gate is realized using magnetic flux to tune the qubit's eigenfrequency - this is accomplished by slowly increasing the magnetic flux, waiting for some time to add a phase $\pi$ and then decreasing the flux back to 0 \[1\]. CZ gate allows for faster two-qubit control than the cross-resonance gate due to stronger inductive coupling. 

The matrix representation of $U_{\rm CZ}$ is:
$$
U_{\rm CZ} = |0\rangle\langle 0| \otimes I + |1\rangle\langle1| \otimes \hat{\sigma}^z = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & -1 \end{bmatrix}. 
$$  

Using the circuit identity, the CNOT gate can be implemented by one CZ gate and two Hadamard gates since $H\hat{\sigma}^zH=\hat{\sigma}^x$ \[1\]. 

![cnot](figures/cz-cnot.png)

## Construct  Hamiltonian

In our modeling, we will account for energy leakage for a two-qubit system by including the third energy level. In the rotating frame, the system Hamiltonian is defined as \[1\]:

$$
\hat{H}_{\rm sys}(t) = (\omega_{\rm q0}-\omega_{\rm d0})\hat{a}_0^\dagger \hat{a}_0+(\omega_{\rm q1}-\omega_{\rm d0}) \hat{a}_1^\dagger \hat{a}_1+\frac{\alpha_0}{2}\hat{a}_0^{\dagger}\hat{a}_0^\dagger\hat{a}_0\hat{a}_0 + \frac{\alpha_1}{2}\hat{a}_1^\dagger\hat{a}_1^\dagger\hat{a}_1\hat{a}_1 + \frac{g}{2}(\hat{a}_0\hat{a}_1^\dagger+\hat{a}_0^\dagger\hat{a}_1)+\frac{A_0^z(t)}{2}\hat{a}_0^\dagger \hat{a}_0,
$$

where $\hat{a}_i^\dagger$, $\hat{a}_i$ are the creation and annihilation operators for qubit i (i=0,1). The information regarding the hardware structure is specified by parameters: qubit frequency $\omega_{qi}$, drive frequency $\omega_{{\rm d}i}$, anharmonicity $\alpha_i$, and coupling strength $g$.  

In Quanlse, we construct the system Hamiltonian by adding up the three terms:
$$
\hat{H}_{\rm sys}(t) = \hat{H}_{\rm drift} + \hat{H}_{\rm coup} + \hat{H}_{\rm ctrl}(t). 
$$

We first define the necessary function arguments, including the sampling period, number of qubits, and the system's energy levels to consider. We then use the function `qham.createHam()` to initialize the Hamiltonian dictionary by passing in the variables we just defined.

In [ ]:
# Sampling period
dt = 0.2

# Number of qubits
qubits = 2

# System energy levels
level = 3

# Initilize the Hamiltonian
ham = qham.createHam(title='ctrl-z', dt=dt, qubitNum=qubits, sysLevel=level) # Initialize the system Hamiltonian

After initializing the Hamiltonian dictionary, we start to add terms to the Hamiltonian. Here, we define the parameters of the hardware that will be used later:

In [ ]:
# Specify the parameters of the hardware
g = 0.0277 * (2 * pi) # Coupling strength, GHz
wq0 = 5.287 * (2 * pi) # The qubit frequency for qubit 0, GHz
wq1 = 4.662 * (2 * pi) # The qubit frequency for qubit 1, GHz
wd0 = wq0              # The drive frequency, GHz
anharm0 = -0.317 * (2 * pi) # Anharmonicity for qubit 0, GHz
anharm1 = -0.226 * (2 * pi) # Anharmonicity for qubit 1, GHz
detuning0 = wq0 - wd0  
detuning1 = wq1 - wd0

Next, we add the drift terms to the initalized Hamiltonian. The drift Hamiltonian $\hat{H}_{\rm drift}$ takes the form:
$$
\hat{H}_{\rm drift} = (\omega_{\rm q0}-\omega_{\rm d0})\hat{n}_0+(\omega_{\rm q1}-\omega_{\rm d0})\hat{n}_1+\frac{\alpha_0}{2}\hat{a}_0^\dagger\hat{a}_0^\dagger\hat{a}_0\hat{a}_0+\frac{\alpha_1}{2}\hat{a}_1^\dagger\hat{a}_1^\dagger\hat{a}_1\hat{a}_1 .
$$
Here, $\hat{n}_i=\hat{a}^\dagger_i \hat{a}_i$ is the number operator and $\hat{a}_i^\dagger\hat{a}_i^\dagger\hat{a}_i\hat{a}_i$ is the Duffing operator for qubit i (i=0,1). In Quanlse, this can be done using the function `addDrift()` which takes a Hamiltonian dictionary, a user-defined name, the list of qubit(s) the term acts upon, the according operators, and the amplitude:

In [ ]:
# Add the drift terms
qham.addDrift(ham, name='drift1', onQubits=0, matrices=number(level), amp=detuning0)
qham.addDrift(ham, name='drift2', onQubits=1, matrices=number(level), amp=detuning1)
qham.addDrift(ham, name='drift3', onQubits=0, matrices=duff(level), amp=0.5 * anharm0)
qham.addDrift(ham, name='drift4', onQubits=1, matrices=duff(level), amp=0.5 * anharm1)

Now, we add the coupling Hamiltonian $\hat{H}_{\rm coup}$, which takes the form:
$$
\hat{H}_{\rm coup} = \frac{g}{2}(\hat{a}_0\hat{a}_1^\dagger+\hat{a}_0^\dagger\hat{a}_1). 
$$

The coupling Hamiltonian is added by the function `addCoupling()`. The function arguments include a Hamiltonian dictionary, a user-defined name, a list of qubit indices which the term acts upon, and the coupling strength. 

In [ ]:
# Add the coupling term
qham.addCoupling(ham, name='coupling', onQubits=[0, 1], g=g / 2)

The control term related to the magnetic flux on qubit $q_0$ along $z$ axis is:
$$
\hat{H}_{\rm ctrl}(t) = \frac{A_0^z(t)}{2}\hat{a}_0^\dagger \hat{a}_0 = \frac{A_0^z(t)}{2}\hat{n}_0.
$$
We use the function `addControl()` to add this term to the Hamiltonian:

In [ ]:
# Add the control term
qham.addControl(ham, name='q0-ctrlz', onQubits=0, matrices=number(level))

## Generate and optimize pulse via Quanlse Cloud Service

The optimization process usually takes a long time to process on local devices, however, we provide a cloud service that could speed up this process significantly. To use the Quanlse Cloud Service, the users need to acquire a token from http://quantum-hub.baidu.com.

In [ ]:
# Import tools to get access to cloud service
from Quanlse import Define

# To use remoteOptimizerCz() on cloud, paste your token (a string) here
Define.hubToken = ''

For this example, we can use `remoteOptimizeCz()` to generate and optimize the control pulses at the designated `targetInfidelity`. To use this function, the user needs to specify a Hamiltonian dictionary, the amplitude's bound, gate time, maximum iteration number, and the target infidelity. It returns a Hamiltonian and the infidelity found at the local minimum. `aBound` sets the bound for the strength of our pulse - a larger bound will lead to a larger search space. Hence, we can increase the number of iterations to reach a better result.

In [ ]:
aBound=(-5, -1) # The bound of the pulse's strength 
ham, infidelity = remoteOptimizeCz(ham, aBound=aBound, tg=40, maxIter=5, targetInfidelity=0.005)

We now extract the optimized pulse and the infidelity. The gate infidelity for performance assessment throughout this tutorial is defined as ${\rm infid} = 1 - \frac{1}{d}\left|{\rm Tr}[U^\dagger_{\rm goal}P(U)]\right|$, where $d$ is the dimension of $U_{\rm goal}$ and $U$ is the unitary evolution. Note that $P(U)$ in particular describes the evolution projected to the computational subspace.

In [ ]:
print(f"minimum infidelity: {infidelity}")
qham.plotWaves(ham, ['q0-ctrlz'])

The system we defined previously is constituted by two three-level qubits. This indicates that the time evolution operator for this system is a $9\times 9$ matrix. To simulate the evolution, you can use `getUnitary()`:

In [ ]:
unitary = qham.getUnitary(ham)
print('Time evolution operator:\n', round(unitary, 2))

The projected evolution $P(U)$ is as follows:

In [ ]:
process2d = project(unitary, qubits, level, 2)
print("The projected evolution P(U):\n", round(process2d, 2))

## Summary

This tutorial introduces controlled-Z gate's implementation using Quanlse Cloud Service. The users could follow this link [tutorial-cz-gate.ipynb](https://github.com/baidu/Quanlse/tree/master/Tutorial/EN/tutorial-cz.ipynb) to the GitHub page of this Jupyter Notebook document and run this program for themselves. The users are encouraged to try parameter values different from this tutorial to obtain the optimal result.

## References
\[1\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://doi.org/10.1063/1.5089550)